# LAB-2 Data Preparation

# Introduction

Let's suppose that you are working for a bank (not so strange for some of you **:-)** ); this bank wants to offer new services to private individuals. Offered services include account management, loans, credit card services, etc. The bank is interesting in finding groups of clients as targets for the new services and the bank wants to set aside clients with risk of defaulting. Bank managers have a slight idea of who could be targeted (who to offer additional services) and who is not eligible (who to watch carefully to minimize the bank losses). The bank stores data about their clients:
accounts (transactions within several months), loans already granted, credit cards issued... So that some idea of customer behavior can be extracted (and questions as well) by analyzing this data. 

We are going to follow the basic process of loan default prediction with machine learning algorithms.

## Task description for LAB-2 and LAB-3

We will work with 8 datasets extracted from the databases of the bank which, of course are stored on Db2 for z/OS (where else?). As data scientists, we will need to perform the following tasks:

**LAB-2**
* Use Python to connect to Db2 for z/OS and read tables into Pandas dataframes
* Preprocess data for machine learning

**LAB-3**
* Train a ML model to predict customers who are more likely to default on loans
* Evaluate model performance
* Try to understand the key predictors of default

# Beginning of LAB-2: Analysis and data preparation

Through this Jupyter notebook we will load data from some Db2 for z/OS tables and we will work the data contained in them. They are the same tables you used to visualize data in LAB-1. We need to reorder some tables, merge them, drop some columns and fill some gaps. After all this we will end up with a consistent Pandas dataframe suitable to train some machine learning models.

**PLEASE:** Read with attention the instructions in the script you have in `*.pdf` format as well as the text cells in this labs. Our intention is that all the information needed to understand the lab can be found in these sources.

# Exploratory Analysis

To begin this exploratory analysis, we first import libraries. They are sets of Python programs and functions that will make our life easier. We are using some of the most popular packages used for data science with Python:

* Pandas, used to manipulate data in tables called dataframes. 
* Numpy, used for numeric formatting, array management and numeric calculations.
* Seaborn, used for fancy visualization inline with the code.
* Matplotlib, used together with Seaborn.
* Sklearn, contains the algorithms we will use for machine learning.

With all these open source packages and Python tools we can do all the required works to arrange data. 

In [ ]:
# CELL 1

import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import time
from sklearn import metrics
from sklearn import neighbors
from sklearn import ensemble
from sklearn import tree
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from pandas.plotting import scatter_matrix
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from datetime import datetime, date, time, timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
import matplotlib.ticker as mtick
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn import svm
#import scikitplot as skplt
plt.style.use('ggplot')
#_______________________ Added to be able to load files from the Cloud Storage ________________________
import dsx_core_utils, requests, os, io
from dsx_core_utils import ProjectContext
from pyspark.sql import SparkSession
from pyspark import SparkContext

# Data Preparation

## Import and Update tables

We load the Db2 table `CARD` and make some small changes like formatting the date in the `issued` column and the card type into numbers.

In [ ]:
# CELL2

#### Insert code to load CARD data in this cell



In [ ]:
# CELL 3

# changes in two columns: issued and type
card.issued = card.issued.str.strip("00:00:00")
card.type = card.type.map({"gold": 2, "classic": 1, "junior": 0})
card.head()

In [ ]:
# CELL 4

# Load table ACCOUNT

# Add asset from remote connection
account_raw = None
dataSet = dsx_core_utils.get_remote_data_set_info(pc, 'ACCOUNT')
dataSource = dsx_core_utils.get_data_source_info(pc, dataSet['datasource'])
# Load JDBC data to Spark dataframe
dbTableOrQuery = (dataSet['schema'] + '.' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table']
account_raw = sparkSession.read.format("jdbc").option("driver",dataSource['driver_class']).option("url", dataSource['URL']).option("dbtable",dbTableOrQuery).option("user",dataSource['user']).option("password",dsx_core_utils.decrypt(dataSource['password'])).load()

account = account_raw.toPandas()

# Date is formatted applying a lambda function to the column
account.date = account.date.apply(lambda x: pd.to_datetime(str(x), format="%y%m%d"))
account.head()

In [ ]:
# CELL 5    
    
    # Load table DISP

# Add asset from remote connection
disp_raw = None
dataSet = dsx_core_utils.get_remote_data_set_info(pc, 'DISP')
dataSource = dsx_core_utils.get_data_source_info(pc, dataSet['datasource'])
# Load JDBC data to Spark dataframe
dbTableOrQuery = (dataSet['schema'] + '.' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table']
disp_raw = sparkSession.read.format("jdbc").option("driver",dataSource['driver_class']).option("url", dataSource['URL']).option("dbtable",dbTableOrQuery).option("user",dataSource['user']).option("password",dsx_core_utils.decrypt(dataSource['password'])).load()

disp = disp_raw.toPandas()

# We select all rows where the value for type is OWNER then change the name of the column
disp = disp[disp.type == "OWNER"]
disp.rename(columns={"type": "type_disp"}, inplace=True)
disp.head()

In [ ]:
# CELL 6

# Load table CLIENT

# Add asset from remote connection
client_raw = None
dataSet = dsx_core_utils.get_remote_data_set_info(pc, 'CLIENT')
dataSource = dsx_core_utils.get_data_source_info(pc, dataSet['datasource'])
# Load JDBC data to Spark dataframe
dbTableOrQuery = (dataSet['schema'] + '.' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table']
client_raw = sparkSession.read.format("jdbc").option("driver",dataSource['driver_class']).option("url", dataSource['URL']).option("dbtable",dbTableOrQuery).option("user",dataSource['user']).option("password",dsx_core_utils.decrypt(dataSource['password'])).load()

client = client_raw.toPandas()

# This is an interesting extraction of information which is coded in the birth date for customers:
# the number is in the form YYMMDD for men,
# the number is in the form YYMM+50DD for women
# where YYMMDD is the date of birth
# with the following code we extract the sex information from the birth dates and add a new 
# column with the sex: 0 for female, 1 for male.
# Also we drop some columns: birth_number, month, year

client["month"] = client.birth_number.apply(
    lambda x: x // 100 % 100, convert_dtype=True, args=())
client["year"] = client.birth_number.apply(
    lambda x: x // 100 // 100, convert_dtype=True, args=())
client["age"] = 99 - client.year # age in 1999
client["sex"] = client.month.apply(lambda x: (x - 50) < 0, convert_dtype=True, args=()) 
client.sex = client.sex.astype(int)  # 0 for female, 1 for male
client.drop(["birth_number", "month", "year"], axis=1, inplace=True)
client.head()

In [ ]:
# CELL 7

# Load table DISTRICT

# Add asset from remote connection
district_raw = None
dataSet = dsx_core_utils.get_remote_data_set_info(pc, 'DISTRICT')
dataSource = dsx_core_utils.get_data_source_info(pc, dataSet['datasource'])
# Load JDBC data to Spark dataframe
dbTableOrQuery = (dataSet['schema'] + '.' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table']
district_raw = sparkSession.read.format("jdbc").option("driver",dataSource['driver_class']).option("url", dataSource['URL']).option("dbtable",dbTableOrQuery).option("user",dataSource['user']).option("password",dsx_core_utils.decrypt(dataSource['password'])).load()

district = district_raw.toPandas()

# Drop columns A2 and A3
district.drop(["A2", "A3"], axis=1, inplace=True)
district.head()

In [ ]:
# CELL 8

# Load table ORDER

# Add asset from remote connection
order_raw = None
dataSet = dsx_core_utils.get_remote_data_set_info(pc, 'ORDER')
dataSource = dsx_core_utils.get_data_source_info(pc, dataSet['datasource'])
# Load JDBC data to Spark dataframe
dbTableOrQuery = (dataSet['schema'] + '.' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table']
order_raw = sparkSession.read.format("jdbc").option("driver",dataSource['driver_class']).option("url", dataSource['URL']).option("dbtable",dbTableOrQuery).option("user",dataSource['user']).option("password",dsx_core_utils.decrypt(dataSource['password'])).load()

order = order_raw.toPandas()

# This code rearranges the dataframe by dropping some columns, filling blanks and empty records,
# renaming some other columns and resetting the index column.
# Renaming some columns has been done for the sake of clarity in English. The names have been translated
# from Czech to English

order.drop(["bank_to", "account_to", "order_id"], axis=1, inplace=True)
order.k_symbol.fillna("No_symbol")
order.k_symbol = order.k_symbol.str.replace(" ", "No_symbol")
order = order.groupby(["account_id", "k_symbol"]).mean().unstack()
order = order.fillna(0)
order.columns = order.columns.droplevel()
order.reset_index(level="account_id", col_level=1, inplace=True)
order.rename_axis("", axis="columns", inplace=True)
order.rename(
    index=None,
    columns={
        "LEASING": "order_amount_LEASING_PAYMENT",
        "No_symbol": "order_amount_No_symbol",
        "POJISTNE": "order_amount_INSURANCE_PAYMENT",
        "SIPO": "order_amount_HOUSEHOLD_PAYMENT",
        "UVER": "order_amount_LOAN_PAYMENT",
    },
    inplace=True)

order.head()

In [ ]:
# CELL 9

# Load table LOAN

# Add asset from remote connection
loan_raw = None
dataSet = dsx_core_utils.get_remote_data_set_info(pc, 'LOAN')
dataSource = dsx_core_utils.get_data_source_info(pc, dataSet['datasource'])
# Load JDBC data to Spark dataframe
dbTableOrQuery = (dataSet['schema'] + '.' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table']
loan_raw = sparkSession.read.format("jdbc").option("driver",dataSource['driver_class']).option("url", dataSource['URL']).option("dbtable",dbTableOrQuery).option("user",dataSource['user']).option("password",dsx_core_utils.decrypt(dataSource['password'])).load()

# Date formatting using lambda function as before.

loan = loan_raw.toPandas()
loan.date = loan.date.apply(lambda x: pd.to_datetime(str(x), format="%y%m%d"))
loan.head()

In [ ]:
# CELL 10

loan.status.value_counts()

In [ ]:
# CELL 11

# Load table TRANS

# Add asset from remote connection
trans_raw = None
dataSet = dsx_core_utils.get_remote_data_set_info(pc, 'TRANS')
dataSource = dsx_core_utils.get_data_source_info(pc, dataSet['datasource'])
# Load JDBC data to Spark dataframe
dbTableOrQuery = (dataSet['schema'] + '.' if(len(dataSet['schema'].strip()) != 0) else '') + dataSet['table']
trans_raw = sparkSession.read.format("jdbc").option("driver",dataSource['driver_class']).option("url", dataSource['URL']).option("dbtable",dbTableOrQuery).option("user",dataSource['user']).option("password",dsx_core_utils.decrypt(dataSource['password'])).load()

trans = trans_raw.toPandas()

# Merge this table with LOAN after filling all empty records
# Format date column

trans.loc[trans.k_symbol == "", "k_symbol"] = trans[trans.k_symbol == ""].k_symbol.apply(lambda x: "k_symbol_missing")
trans.loc[trans.k_symbol == " ", "k_symbol"] = trans[trans.k_symbol == " "].k_symbol.apply(lambda x: "k_symbol_missing")
loan_account_id = loan.loc[:, ["account_id"]]
trans = loan_account_id.merge(trans, how="left", on="account_id")
trans.date = trans.date.apply(lambda x: pd.to_datetime(str(x), format="%y%m%d"))
trans.head()

Once we have loaded all files with data and made the initial arrangements we start "changing" data into something understandable by the models: creating "features" for the model.

In [ ]:
# CELL 12

# create temp table trans_pv_k_symbol
# Table is pivoted 

trans_pv_k_symbol = trans.pivot_table(values=["amount", "balance"], index=["trans_id"], columns="k_symbol")
trans_pv_k_symbol.fillna(0, inplace=True)
trans_pv_k_symbol.columns = ["_".join(col) for col in trans_pv_k_symbol.columns]
trans_pv_k_symbol = trans_pv_k_symbol.reset_index()
trans_pv_k_symbol = trans.iloc[:, :3].merge(trans_pv_k_symbol, how="left", on="trans_id")
trans_pv_k_symbol.head()

In [ ]:
# CELL 13

# create temp table get_date_loan_trans
# First merge loan and account

get_date_loan_trans = pd.merge(
    loan,
    account,
    how="left",
    on="account_id",
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_loan", "_account"),
    copy=True,
    indicator=False,
    validate=None)

# Then merge the former with loan-account with trans. All of them use account_id as the column guiding the merging.

get_date_loan_trans = pd.merge(
    get_date_loan_trans,
    trans,
    how="left",
    on="account_id",
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_account", "_trans"),
    copy=True,
    indicator=False,
    validate=None)

#### Insert here the head() method to view how merging has configured the new table


In [ ]:
# CELL 14

# update table get_date_loan_trans to get the date between loan_date and trans_date. Then format it to leave a plain number of days.

get_date_loan_trans["date_loan_trans"] =  #### Insert code before this comment ####
get_date_loan_trans[["date_loan_trans"]] = get_date_loan_trans[["date_loan_trans"]].astype(str)

# Format the new column to set the number of days as a number
get_date_loan_trans.date_loan_trans = get_date_loan_trans.date_loan_trans.str.strip(" days 00:00:00.000000000")
get_date_loan_trans.date_loan_trans = pd.to_numeric(get_date_loan_trans.date_loan_trans.str.strip(" days +"))
get_date_loan_trans.head()

In [ ]:
# CELL 15

# create temp table temp_90_mean to create new feature
temp_90_mean = get_date_loan_trans[(get_date_loan_trans["date_loan_trans"] >= 0) & (get_date_loan_trans["date_loan_trans"] < 90)]
temp_90_mean = temp_90_mean.drop(["trans_id", "k_symbol"], axis=1)
temp_90_mean = temp_90_mean.groupby(["loan_id"], as_index=None).mean()
temp_90_mean = temp_90_mean.loc[:, ["loan_id", "balance"]]
temp_90_mean.rename(index=None, columns={"balance": "avg_balance_3M_before_loan"}, inplace=True)

#### Insert code to view the 7 last lines of the dataframe


In [ ]:
# CELL 16

# create temp table temp_30_mean to create new feature
temp_30_mean = get_date_loan_trans[(get_date_loan_trans["date_loan_trans"] >= 0) & (get_date_loan_trans["date_loan_trans"] < 30)]
temp_30_mean = temp_30_mean.drop(["trans_id", "k_symbol"], axis=1)
temp_30_mean = temp_30_mean.groupby(["loan_id"], as_index=None).mean()
temp_30_mean = temp_30_mean.loc[:, ["loan_id", "balance"]]
temp_30_mean.rename(index=None, columns={"balance": "avg_balance_1M_before_loan"}, inplace=True)

#### Insert code to view the 7 last lines of the dataframe


In [ ]:
# CELL 17

# create temp table temp_trans_freq to create new feature.
# A frequency of movements is created
temp_before = get_date_loan_trans[(get_date_loan_trans["date_loan_trans"] >= 0)]
temp_trans_freq = (temp_before.loc[:, ["loan_id", "trans_id"]].groupby(["loan_id"], as_index=None).count())
temp_trans_freq.rename(index=None, columns={"trans_id": "trans_freq"}, inplace=True)
temp_before = temp_before.drop(["trans_id", "k_symbol"], axis=1)

In [ ]:
# CELL 18

# create temp table temp_balance_min & temp_balance_mean to create new features
# Minimun and average balances are added for each of the loans

temp_balance_min = (
    temp_before.groupby(["loan_id"], as_index=None).min().loc[:, ["loan_id", "balance"]]
)
temp_balance_min.rename(
    index=None, columns={"balance": "min_balance_before_loan"}, inplace=True
)

temp_balance_mean = (
    temp_before.groupby(["loan_id"], as_index=None)
    .mean()
    .loc[:, ["loan_id", "amount_trans", "balance"]]
)
temp_balance_mean.rename(
    index=None,
    columns={
        "amount_trans": "avg_amount_trans_before_loan",
        "balance": "avg_balance_before_loan",
    },
    inplace=True,
)

temp_balance_mean.head()

In [ ]:
# CELL 19

# create temp table times_balance_below_500 & times_balance_below_5K to create new features

times_balance_below_500 = temp_before[temp_before.balance < 500]
times_balance_below_500 = (
    times_balance_below_500.groupby(["loan_id"], as_index=None)
    .count()
    .loc[:, ["loan_id", "balance"]]
)
times_balance_below_500 = times_balance_below_500[times_balance_below_500.balance > 1]
times_balance_below_500.rename(
    index=str, columns={"balance": "times_balance_below_500"}, inplace=True
)

times_balance_below_5K = temp_before[temp_before.balance < 5000]
times_balance_below_5K = (
    times_balance_below_5K.groupby(["loan_id"], as_index=None)
    .count()
    .loc[:, ["loan_id", "balance"]]
)
times_balance_below_5K = times_balance_below_5K[times_balance_below_5K.balance > 1]
times_balance_below_5K.rename(
    index=str, columns={"balance": "times_balance_below_5K"}, inplace=True
)


In [ ]:
# CELL 20

# create temp table merge_loan_trans to merge the temp features above into one temp table
merge_loan_trans = loan.merge(
    temp_90_mean, how="left", on="loan_id", suffixes=("_loan", "_trans")
)
merge_loan_trans = merge_loan_trans.merge(temp_30_mean, how="left", on="loan_id")
merge_loan_trans = merge_loan_trans.merge(temp_trans_freq, how="left", on="loan_id")
merge_loan_trans = merge_loan_trans.merge(temp_balance_min, how="left", on="loan_id")
merge_loan_trans = merge_loan_trans.merge(temp_balance_mean, how="left", on="loan_id")
merge_loan_trans = merge_loan_trans.merge(
    times_balance_below_500, how="left", on="loan_id"
)

#### Insert code below to merge the table times_balance_below_5K into the table merge_loan_trans on the left and using loan_id as a guide



#### Visualize the resulting dataframe using head() method



In [ ]:
# CELL 21

# | (pipe) operator is the union of sets when used with dataframes

loan_BorD = loan[(loan.status == "D") | (loan.status == "B")]

#### Insert code below. Need to know how maby rows are there after subsetting D and B status.



In [ ]:
# CELL 22

temp = times_balance_below_500.merge(
    loan,
    how="inner",
    on="loan_id",
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None)

#### Insert code below


In [ ]:
# CELL 23

#### Insert code below to plot status vs times_balance_below_500


In [ ]:
# CELL 24

# We get different views to try to understand the behaviour of payments and balances

temp.sort_values("times_balance_below_500", ascending=False).plot(x="status", y="times_balance_below_500", kind="bar")

In [ ]:
# CELL 25

#### Insert code below. Select all rows but just two columns: payments and status. Check that it's correct viewing the 3 first rows



In [ ]:
# CELL 26

# Group the 4 status and take a mean of the payments made in each of the four groups

t = t.groupby(["status"], as_index=None).mean()

#### Insert code below. Plot the former calculation with status in x axis 



# Merge tables

At this point we are going to create a version of the dataframe that will be used in LAB-3 to train our machine learning models. To do this we are merging some of the original tables, loaded from Db2 for z/OS (with the arrangements done at load time), with some of the dataframes created along this notebook.

In [ ]:
# CELL 27

# Merge the created merge_loan_trans with account. Have a look to the options used to merge.

df = pd.merge(
    merge_loan_trans,
    account,
    how="left",
    on="account_id",
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_loan", "_account"),
    copy=True,
    indicator=False,
    validate=None)


In [ ]:
# CELL 28

# Add order to the new dataframe df

#### Insert code below. Merge the already created dataframe, df, with the dataframe for the ACCOUNT table, whose name is account too.



In [ ]:
# CELL 29

# Add disp

df = pd.merge(
    df,
    disp,
    how="left",
    on="account_id",
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_b", "_disp"),
    copy=True,
    indicator=False,
    validate=None)


In [ ]:
# CELL 30

# Add card

df = pd.merge(
    df,
    card,
    how="left",
    on="disp_id",
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_c", "_card"),
    copy=True,
    indicator=False,
    validate=None)


In [ ]:
# CELL 31

# Add client

#### Insert code below. Merge df with client.



In [ ]:
# CELL 32

# Add district

#### Insert code below. Merge df with district.



In [ ]:
# CELL 33

# Some arrangements before merging with df. We use before_loan_date and trans_pv_k_symbol after some selection from before_loan_date

before_loan_date = get_date_loan_trans[(get_date_loan_trans["date_loan_trans"] >= 0)]
before_loan_date = before_loan_date.loc[:, ["account_id", "trans_id"]]
trans_pv_k_symbol = pd.merge(
    before_loan_date,
    trans_pv_k_symbol,
    how="left",
    on="trans_id",
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_before", "_df2"),
    copy=True,
    indicator=False,
    validate=None)

trans_pv_k_symbol.drop(["account_id_df2", "date", "trans_id"], axis=1, inplace=True)

trans_pv_k_symbol.rename(columns={"account_id_before": "account_id"}, inplace=True)

trans_pv_k_symbol = trans_pv_k_symbol.groupby(by="account_id", axis=0, as_index=False, sort=True, group_keys=True, squeeze=False).mean()


In [ ]:
# CELL 34

# And with this merge we have our final df

#### Insert code below. Merge df with trans_pv_k_symbol




# Data Cleaning

As seen before this final df dataframe needs some final cleaning and tyding up. Pay attention to the next group of cells and study what are these final changes affecting data. Feel free to insert code to check the status of the dataframe in any of the cells where manipulations are done.

In [ ]:
# CELL 35

df["year_"] = df.date_loan.apply(lambda x: x.year, convert_dtype=int, args=())
df["years_of_loan"] = 1999 - df.year_
df.drop(["date_loan", "year_"], axis=1, inplace=True)
df.frequency = df.frequency.map(
    {"MONTHLY ISSUANCE": 30, "WEEKLY ISSUANCE": 7, "ISSUANCE AFTER TRANSACTION": 1}
)

# Insert visualization code here if needed


In [ ]:
# CELL 36

df["year_"] = df.date_account.apply(lambda x: x.year, convert_dtype=int, args=())
df["years_of_account"] = 1999 - df.year_
df.drop(["date_account", "year_", "type_disp"], axis=1, inplace=True)

# Insert visualization code here if needed


In [ ]:
# CELL 37

df.issued.fillna("999999", inplace=True)
df["years_card_issued"] = df.issued.apply(
    lambda x: (99 - int(x[:2])), convert_dtype=int
)
df.drop(["issued","A12","A15"], axis=1, inplace=True)

# Insert visualization code here if needed


In [ ]:
# CELL 38

df.fillna(0, inplace=True)

# Insert visualization code here if needed


In [ ]:
# CELL 39

df.status.value_counts()


In [ ]:
# CELL 40

df.info()


# Get Label

In [ ]:
# CELL 41

# Change categorical variables into numerical ones. There are ML model not admitting categoricals

m = {"A": 0, "B": 1, "C": 0, "D": 1}
df.status = df.status.map(m)

#### Insert code below. Check that the unique values are 0 and 1


In [ ]:
# CELL 42

df = pd.get_dummies(df, drop_first=True)


In [ ]:
# CELL 43

df.columns.unique()

In [ ]:
# CELL 44

#### Insert code below. Drop columns 



# Save the shining df dataframe into a CSV file to reuse it as needed

df.to_csv('fullDataframe_EOLAB1_<login_userid>.csv', sep=';', encoding='utf-8',index = False)

**We finally have a consistent dataframe which is suitable for model training.**

In [ ]:
# CELL 45

pwd